In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
v = '3'
mn= 'torch.20220102-075827'
model_dir = f'../working/online/{v}/{mn}'

In [3]:
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer
from src import config
from src.torch_model import Model
from src.util import get_preds
import melt as mt
from datasets import Dataset
import torch
import lele
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
config.init_()
MAX_LEN = FLAGS.max_len
tokenizer = AutoTokenizer.from_pretrained(FLAGS.backbone, add_prefix_space=True)
model = Model()

ic| config.py:73 in init_()
    FLAGS.backbone: '../input/allenai/longformer-large-4096'
ic| config.py:79 in init_()- FLAGS.num_classes: 15
Some weights of the model checkpoint at ../input/allenai/longformer-large-4096 were not used when initializing LongformerModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at ../input/allenai/longformer-large-4096 and are newly initialized: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
You should 

In [4]:
lele.load(model, f'{model_dir}/model.pt')

{'epoch': 3,
 'step': 2925,
 'state_dict': OrderedDict([('backbone.embeddings.position_ids',
               tensor([[   0,    1,    2,  ..., 4095, 4096, 4097]])),
              ('backbone.embeddings.word_embeddings.weight',
               tensor([[-1.2620e-01, -4.2589e-03,  3.6409e-02,  ...,  4.8356e-02,
                        -1.2965e-05, -3.8844e-02],
                       [ 7.7477e-03, -1.5526e-02,  1.5503e-02,  ..., -1.5518e-02,
                         2.2903e-02,  1.5526e-02],
                       [-7.4972e-02,  1.0324e-03, -1.1500e-01,  ...,  1.0446e-01,
                         6.5437e-02, -4.0188e-02],
                       ...,
                       [ 4.0131e-02,  1.6527e-03,  4.6051e-02,  ..., -2.4597e-02,
                        -4.8889e-02,  3.3844e-02],
                       [ 4.8706e-02,  2.6596e-02,  3.9307e-02,  ..., -3.5919e-02,
                        -1.1147e-02,  8.2779e-03],
                       [-1.5625e-02, -1.1894e-02, -2.3956e-02,  ...,  4.4464e-02,
 

In [5]:
# GET TEST TEXT IDS
files = os.listdir('../input/feedback-prize-2021/test')
TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
print('There are',len(TEST_IDS),'test texts.')

There are 5 test texts.


In [6]:
# CONVERT TEST TEXT TO TOKENS
test_tokens = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')
test_attention = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')
word_ids_list = []
for id_num in tqdm(range(len(TEST_IDS))):
    # READ TRAIN TEXT, TOKENIZE, AND SAVE IN TOKEN ARRAYS    
    n = TEST_IDS[id_num]
    # name = f'../input/feedback-prize-2021/train/{n}.txt'
    name = f'../input/feedback-prize-2021/test/{n}.txt'
    txt = open(name, 'r').read()
    encoding = tokenizer(txt.split(),
                        is_split_into_words=True,
                        padding='max_length', 
                        truncation=True, 
                        max_length=MAX_LEN)
    test_tokens[id_num,] = encoding['input_ids']
    test_attention[id_num,] = encoding['attention_mask']
    word_ids = encoding.word_ids()
    word_ids = [-1 if x is None else x for x in word_ids]
    word_ids_list.append(word_ids)

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
 dataset = Dataset.from_dict({
   'input_ids': test_tokens,
   'attention_mask': test_attention
 })

In [8]:
dataset.set_format(type='torch')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

In [9]:
p = lele.predict(model, dataloader)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# INFER TEST TEXTS
test_preds = np.argmax(p['pred'],axis=-1)
scores = gezi.softmax(p['pred'], axis=-1)

In [11]:
list(p.keys())

['pred']

In [12]:
test_preds

array([[10, 10,  9, ...,  3,  3,  3],
       [10, 10,  9, ...,  3,  3,  3],
       [10, 10,  9, ...,  3,  3,  3],
       [ 0, 10,  9, ...,  3,  3,  3],
       [10, 10,  9, ...,  3,  3,  3]])

In [13]:
if 'start_logits' in  p:
  test_preds = (test_preds, gezi.softmax(p['start_logits'], axis=-1))
df = get_preds(TEST_IDS, test_preds, word_ids_list, scores)

  0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
df

,id,class,predictionstring
0,D46BCB48440A,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
1,D46BCB48440A,Position,20 21 22 23 24 25 26 27
2,D46BCB48440A,Claim,28 37 38
3,D46BCB48440A,Claim,39 40 41 42
4,D46BCB48440A,Evidence,43 45 57 58 59 60 61 62 63 64 65 66 67 68 69 7...
5,D46BCB48440A,Evidence,139 151 152 153 154 155 156 157 158 159 160 16...
6,D46BCB48440A,Evidence,213 224 225 226 227 228 229 230 231 232 233 23...
7,D46BCB48440A,Concluding Statement,306 307 308 309 310 311 312 313 314 315 316 31...
8,DF920E0A7337,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
9,DF920E0A7337,Position,68 69 70 71 72 73 74 75 76 77 78 79 80 81 82


In [ ]:
df.to_csv(f'{model_dir}/submission.csv')